# Desafio: 

Você trabalha em uma grande empresa de Cartão de Crédito e o diretor da empresa percebeu que o número de clientes que cancelam seus cartões tem aumentado significativamente, causando prejuízos enormes para a empresa

O que fazer para evitar isso? Como saber as pessoas que têm maior tendência a cancelar o cartão?

# O que temos:

Temos 1 base de dados com informações dos clientes, tanto clientes atuais quanto clientes que cancelaram o cartão

Download da Base de Dados: https://drive.google.com/file/d/1c0F7BDwvOZ9NnIj3tUANuvmp_jRYsh31/view?usp=sharing

Referência: https://www.kaggle.com/sakshigoyal7/credit-card-customers

Passo 1: Importar a base de dados

Passo 2: Visualizar a base de dados e ver se precisa de algum "tratamento"
  - Excluir/Corrigir linhas e valores vazios
  - Ajeitar colunas importadas da forma errada (ex: Moedas, caso haja problema)
  - Excluir colunas inúteis

Passo 3: Analisar a base de dados de acordo com o que queremos descobrir
  - Queremos descobrir as principais causas de cancelamento ou quem são os clientes com maior chance de cancelar o cartão, para podermos criar um plano de ação

### Importando a base de Dados

In [2]:
import pandas as pd

clientes_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projetos Intensivão de Python/Aula 2/ClientesBanco.csv', encoding='latin1')
clientes_df = clientes_df.drop('CLIENTNUM', axis=1)
display(clientes_df)

Categoria  ...  Taxa de Utilização Cartão
0        Cliente  ...                      0.061
1        Cliente  ...                      0.105
2        Cliente  ...                      0.000
3        Cliente  ...                      0.760
4        Cliente  ...                      0.000
...          ...  ...                        ...
10122    Cliente  ...                      0.462
10123  Cancelado  ...                      0.511
10124  Cancelado  ...                      0.000
10125  Cancelado  ...                      0.000
10126  Cancelado  ...                      0.189

[10127 rows x 20 columns]

### Tratamentos e visão geral dos dados

In [3]:
clientes_df = clientes_df.dropna()
display(clientes_df.info())
display(clientes_df.describe())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10126 entries, 0 to 10126
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Categoria                      10126 non-null  object 
 1   Idade                          10126 non-null  int64  
 2   Sexo                           10126 non-null  object 
 3   Dependentes                    10126 non-null  int64  
 4   Educação                       10126 non-null  object 
 5   Estado Civil                   10126 non-null  object 
 6   Faixa Salarial Anual           10126 non-null  object 
 7   Categoria Cartão               10126 non-null  object 
 8   Meses como Cliente             10126 non-null  int64  
 9   Produtos Contratados           10126 non-null  int64  
 10  Inatividade 12m                10126 non-null  int64  
 11  Contatos 12m                   10126 non-null  int64  
 12  Limite                         10126 non-null 

None

Idade  ...  Taxa de Utilização Cartão
count  10126.000000  ...               10126.000000
mean      46.327079  ...                   0.274901
std        8.016420  ...                   0.275704
min       26.000000  ...                   0.000000
25%       41.000000  ...                   0.023000
50%       46.000000  ...                   0.175500
75%       52.000000  ...                   0.503000
max       73.000000  ...                   0.999000

[8 rows x 14 columns]

### Agora vamos analisar a base, como estão divididos os clientes e os cancelados 

In [4]:
display(clientes_df['Categoria'].value_counts())
display(clientes_df['Categoria'].value_counts(normalize=True))

Cliente      8499
Cancelado    1627
Name: Categoria, dtype: int64

Cliente      0.839325
Cancelado    0.160675
Name: Categoria, dtype: float64

### Temos vários "approaches" possíveis para resolver o problema.

- Podemos visualizar como as categorias estão divididas entre Clientes e Cancelados

- Podemos olhar apenas os Cancelados e olhar como eles estão divididos por categoria, para identificar oportunidades

In [15]:
import plotly.express as px

# para edições nos gráficos: https://plotly.com/python/histograms/

def grafico_coluna_categoria(coluna, tabela):
  fig = px.histogram(tabela, x=coluna, color='Categoria') # color_discrete_sequence=['#e6ad12', '#02b013']
  fig.show()

for coluna in clientes_df:
  grafico_coluna_categoria(coluna, clientes_df)

# O que conseguimos perceber analisando esses gráficos?

1 - Olhando Taxa de Utilização e Qtde de Transações nos Últimos 12m vemos que quem utiliza menos o cartão tende a cancelar mais (se a pessoa tiver mais de 80 transações nos últimos 12 meses, a chance dele cancelar cai bizarramente)

(O Valor parece ter algo parecido)

2 - Parece que quanto mais vezes a pessoa entrou em contato com a gente, mais ela tende a cancelar. Será que esses contatos são problemas/dificuldades de uso que não estão sendo resolvidos?

3 - Quase a totalidade dos clientes que cancelam são da categoria Blue, talvez a categoria não seja vantajosa para o cliente. É perfil do cliente? Como estão os concorrentes nessa mesma categoria?

# Indo mais a fundo

Podemos agora combinar as features dos nossos palpites para onde deve ser o nosso foco, seguindo a lógica 80-20

Qual a menor quantidade de problemas que conseguimos atacar que vão gerar o maior resultado?

Temos quase a totalidade dos Cancelados na bandeira "Blue", então já sabemos que nosso esforço deve ser focado ali

Mas vamos agora olhar mais a fundo:
  - Quantidade de Transações nos últimos 12m (ou valor das transações)
  - Quantidade de Contatos

In [7]:
cancelados_df = clientes_df.loc[clientes_df['Categoria'] == 'Cancelado', :]

fig = px.histogram(cancelados_df, x='Qtde Transacoes 12m', nbins=5)
fig.show()

fig = px.histogram(cancelados_df, x="Valor Transacoes 12m", nbins=7)
fig.show()

In [10]:
criticos_df = cancelados_df.loc[cancelados_df['Qtde Transacoes 12m'] < 60, :]
fig = px.histogram(criticos_df, x='Contatos 12m')
fig.show()

In [11]:
qtde_ultra_criticos = len(criticos_df.loc[criticos_df['Contatos 12m'] > 2, :])

percentual_criticos = qtde_ultra_criticos / len(cancelados_df)
print(f'{percentual_criticos:.1%}')

57.8%


Comparação com um projeto usando Machine Learning: https://www.kaggle.com/alpertml/credit-card-customers-eda-ml-97-5-accuracy

# Outro approach:

Vamos ver como os Cancelados estão divididos ao todo

In [ ]:


for coluna in cancelados_df:
  grafico_coluna_categoria(coluna, cancelados_df)